# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv("city_dataframe.csv")
file.head()


,City ID,City Name,Country,Cloudiness,Max Temp,Wind Speed,Humidity,Lat,Lng
0,0,Punta Arenas,CL,0,51.80,18.41,58,-53.1500,-70.9167
1,1,Butaritari,KI,70,81.88,14.97,77,3.0707,172.7902
2,2,Bluff,NZ,42,57.99,8.01,74,-46.6000,168.3333
3,3,Kirakira,SB,96,80.92,6.22,81,-10.4544,161.9205
4,4,Airai,TL,98,64.51,0.94,97,-8.9266,125.4092


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"]


In [5]:
# heat map
fig = gmaps.figure(center = [0,0], zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                  point_radius = 4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to find your ideal weather condition.
# A max temperature lower than 70 degrees but higher than 60.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_weather_df = file.loc[(file["Max Temp"] >=60) & (file["Max Temp"] <=70) & (file["Wind Speed"] <10) & (file["Cloudiness"] ==0)].dropna()

ideal_weather_df


,City ID,City Name,Country,Cloudiness,Max Temp,Wind Speed,Humidity,Lat,Lng
32,32,Los Llanos de Aridane,ES,0,62.60,4.61,72,28.6585,-17.9182
51,51,Hong Kong,HK,0,61.00,8.61,75,22.2855,114.1577
114,114,Salalah,OM,0,69.80,4.61,56,17.0151,54.0924
525,525,Biltine,TD,0,69.62,7.14,26,14.5333,20.9167


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df

,City ID,City Name,Country,Cloudiness,Max Temp,Wind Speed,Humidity,Lat,Lng,Hotel Name
0,32,Los Llanos de Aridane,ES,0,62.60,4.61,72,28.6585,-17.9182,
1,51,Hong Kong,HK,0,61.00,8.61,75,22.2855,114.1577,
2,114,Salalah,OM,0,69.80,4.61,56,17.0151,54.0924,
3,525,Biltine,TD,0,69.62,7.14,26,14.5333,20.9167,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel", 
         "keyword": "hotel", 
         "radius": "5000",
         "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    City_Name = row["City Name"]
    
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    print(f"Retrieving Results for Index {index}: {City_Name}")
    
    results = response["results"]
    
 
    try:
        print(f"Closest hotel in {City_Name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing result...skipping.")
        
    print("------------------------------------------------------------------------")
  
    time.sleep(1)

Retrieving Results for Index 0: Los Llanos de Aridane
Closest hotel in Los Llanos de Aridane is Hotel Hacienda de Abajo.
------------------------------------------------------------------------
Retrieving Results for Index 1: Hong Kong
Closest hotel in Hong Kong is The Harbourview.
------------------------------------------------------------------------
Retrieving Results for Index 2: Salalah
Closest hotel in Salalah is HYATT SALALAH HOTEL.
------------------------------------------------------------------------
Retrieving Results for Index 3: Biltine
Missing result...skipping.
------------------------------------------------------------------------


In [9]:
#updated hotel_df with hotel name
hotel_df

,City ID,City Name,Country,Cloudiness,Max Temp,Wind Speed,Humidity,Lat,Lng,Hotel Name
0,32,Los Llanos de Aridane,ES,0,62.60,4.61,72,28.6585,-17.9182,Hotel Hacienda de Abajo
1,51,Hong Kong,HK,0,61.00,8.61,75,22.2855,114.1577,The Harbourview
2,114,Salalah,OM,0,69.80,4.61,56,17.0151,54.0924,HYATT SALALAH HOTEL
3,525,Biltine,TD,0,69.62,7.14,26,14.5333,20.9167,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

## not working 

Figure(layout=FigureLayout(height='420px'))